In [ ]:
%reload_ext autoreload
%autoreload 2
    
import numpy as np
import pandas as pd
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42 # for pdfs
import matplotlib.pyplot as plt
from matplotlib import cm
from pathlib import Path
import seaborn as sns

import flexiznam as flz
from cottage_analysis.analysis import spheres, common_utils
from cottage_analysis.pipelines import pipeline_utils
from v1_depth_analysis.v1_manuscript_2023 import depth_selectivity, rf, get_session_list, roi_location

In [ ]:
VERSION = 6
SAVE_ROOT = Path(
    f"/camp/lab/znamenskiyp/home/shared/presentations/v1_manuscript_2023/ver{VERSION}"
)
SAVE_ROOT.mkdir(parents=True, exist_ok=True)

In [ ]:
# Load data
# Load data for example reconstructed stimulus frame
project = "hey2_3d-vision_foodres_20220101"
flexilims_session = flz.get_flexilims_session(project)

In [ ]:
# Load data for example RF
session_name_rf = "PZAH8.2h_S20230116"
vs_df_rf, trials_df_rf = spheres.sync_all_recordings(
    session_name=session_name_rf,
    flexilims_session=flexilims_session,
    filter_datasets={"anatomical_only": 3},
    recording_type="two_photon",
    protocol_base="SpheresPermTubeReward",
    photodiode_protocol=5,
    return_volumes=True,
)
frames_rf, imaging_df_rf = spheres.regenerate_frames_all_recordings(
    session_name=session_name_rf,
    flexilims_session=flexilims_session,
    filter_datasets={"anatomical_only": 3},
    recording_type="two_photon",
    protocol_base="SpheresPermTubeReward",
    photodiode_protocol=5,
    return_volumes=True,
    resolution=1,
)
neurons_ds_rf = pipeline_utils.create_neurons_ds(
    session_name=session_name_rf,
    flexilims_session=flexilims_session,
    project=project,
    conflicts="skip",
)
neurons_df_rf = pd.read_pickle(neurons_ds_rf.path_full)

In [ ]:
# # # Load data for significant RF %
# project = "hey2_3d-vision_foodres_20220101"
# flexilims_session = flz.get_flexilims_session(project)
# all_sig, all_sig_ipsi, neurons_df_all = rf.load_sig_rf(
#     flexilims_session=flexilims_session,
#     session_list=get_session_list.get_sessions(
#         flexilims_session=flexilims_session,
#         exclude_openloop=False,
#         exclude_pure_closedloop=False,
#         v1_only=True,
#     ),
#     n_std=6,
# )

# neurons_df_all = roi_location.align_across_mice(neurons_df_all)
# neurons_df_all.to_pickle(SAVE_ROOT / "rf_neurons_df_all.pkl")

# import pickle
# with open(SAVE_ROOT / "rf_all_sig.pkl", "wb") as f:
#     pickle.dump(all_sig, f, protocol=pickle.HIGHEST_PROTOCOL)
# with open(SAVE_ROOT / "rf_all_sig_ipsi.pkl", "wb") as f:
#     pickle.dump(all_sig_ipsi, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle
neurons_df_all = pd.read_pickle(SAVE_ROOT/"rf_neurons_df_all.pkl")
all_sig = pickle.load(open(SAVE_ROOT / "rf_all_sig.pkl", "rb"))
all_sig_ipsi = pickle.load(open(SAVE_ROOT / "rf_all_sig_ipsi.pkl", "rb"))

In [ ]:
# session_df_all = rf.calculate_rf_gradient_all_sessions(flexilims_session,
#                                        session_list=neurons_df_all.session.unique(),
#                                        neurons_df_all_aligned = neurons_df_all,
#                                        filename='rf_gradients.pkl',
#                                        conflicts='skip',
#                                        verbose=True,
#                                        )
# session_df_all.to_pickle(SAVE_ROOT / "rf_gradients_all_sessions.pkl")
session_df_all = pd.read_pickle(SAVE_ROOT / "rf_gradients_all_sessions.pkl")
session_df_all["azi_angle"] = np.degrees(np.arctan2(session_df_all["slope_y_azi_rf_sig"], -session_df_all["slope_x_azi_rf_sig"]))
session_df_all["ele_angle"] = np.degrees(np.arctan2(session_df_all["slope_y_ele_rf_sig"], -session_df_all["slope_x_ele_rf_sig"]))

## 3D RF


In [ ]:
fontsize_dict = {"title": 7, "label": 7, "tick": 6, "legend": 5}
fig = plt.figure(figsize=(18 / 2.54, 18 / 2.54))
# Fig.2A Example reconstructed stimuli frame
trial_idx = 4
example_trial = trials_df_rf.iloc[trial_idx]
depths = np.sort(trials_df_rf.depth.unique())
idepth = np.nonzero(depths == example_trial.depth)[0][0]
iframe = (trials_df_rf.iloc[trial_idx].imaging_stim_start + 10,)
rf.plot_stimulus_frame(
    frame=frames_rf[iframe, :, frames_rf.shape[2] // 2 :].squeeze(),
    idepth=idepth,
    depths=depths,
    position=[0, 0.74, 0.3, 0.4],
    fontsize_dict=fontsize_dict,
    plot_prop=0.9,
)

# Fig.2B Linear-nonlinear receptive field model schematic. (maybe not necessary)

# Fig.2C Example receptive field of 2 V1 neurons
# PZAH8.2h_S20230116: 271, 299, 319, 321, 402, 344, 356, 306, 110, 117, 86, 88, 93, 110, 117, 120, 141, 223, 250, 350, 370, 412, 434, 437
SELECT_ROIS = [
    319,
    720,
    120,
]  # change 271 to 591
for iroi, (roi, linecolor) in enumerate(zip(SELECT_ROIS, ["red", "green", "blue"])):
    fig.add_axes([0.267 + 0.165 * iroi, 0.9, 0.05, 0.05])
    depth_selectivity.plot_depth_tuning_curve(
        neurons_df=neurons_df_rf,
        trials_df=trials_df_rf,
        roi=roi,
        rs_thr=None,
        plot_fit=True,
        plot_smooth=False,
        linewidth=1.5,
        linecolor=linecolor,
        markersize=1.5,
        closed_loop=1,
        fontsize_dict=fontsize_dict,
    )
    plt.ylabel("")
    plt.xlabel("")
    plt.gca().set_xticklabels([])
    plt.gca().tick_params(length=1.5)
    if iroi == len(SELECT_ROIS) // 2:
        xlabel = "Azimuth (degrees)"
    else:
        xlabel = ""
    if iroi == 0:
        ylabel = "Elevation (degrees)"
    else:
        ylabel = ""
    rf.plot_rf(
        neurons_df=neurons_df_rf,
        roi=roi,
        ndepths=8,
        frame_shape=(16, 24),
        position=[0.055 + 0.165 * iroi, 0.828, 0.5, 0.5],
        plot_prop=0.9,
        xlabel=xlabel,
        ylabel=ylabel,
        fontsize_dict=fontsize_dict,
    )
plt.savefig(SAVE_ROOT / "fig2a.pdf", dpi=300)
plt.savefig(SAVE_ROOT / "fig2a.png", dpi=300)

In [ ]:
rf.plot_rf_3d(
    neurons_df_rf,
    SELECT_ROIS,
    depths,
    SAVE_ROOT / "fig2_3d_rfs.pdf",
    dict(label=10, tick=10),
)

In [ ]:
# Fig.2D Proportion of neurons with significant receptive fields across imaging sessions
fontsize_dict = {"title": 7, "label": 7, "tick": 6, "legend": 5}
fig = plt.figure(figsize=(18 / 2.54, 18 / 2.54))
fig.add_axes([0.05, 0.7, 0.13, 0.13])
rf.plot_sig_rf_perc(
    all_sig=all_sig,
    all_sig_ipsi=all_sig_ipsi,
    plot_type="hist",
    bar_color="k",
    bins=np.arange(0, 1, 0.1),
    fontsize_dict=fontsize_dict,
)

# Fig.2F-G Preferred azimuth & elevation follow V1 retinotopic gradients
# example session, spatial distribution of preferred azimuth/elevation/depth
# visualize azimuth & elevation on the 2p cell mask (or as a function of position)--> color-code - two separate plots --> include position in neurons_df
# Load suite2p ops files
suite2p_ds = flz.get_datasets(
    flexilims_session=flexilims_session,
    origin_name=session_name_rf,
    dataset_type="suite2p_rois",
    filter_datasets={"anatomical_only": 3},
    allow_multiple=False,
    return_dataseries=False,
)
iscell = np.load(suite2p_ds.path_full / "plane0" / "iscell.npy", allow_pickle=True)[
    :, 0
]
neurons_df_rf["iscell"] = iscell
stat = np.load(suite2p_ds.path_full / "plane0" / "stat.npy", allow_pickle=True)
ops = np.load(suite2p_ds.path_full / "plane0" / "ops.npy", allow_pickle=True).item()

fig.add_axes([0.05, 0.45, 0.1, 0.1])
depth_selectivity.plot_fov_mean_img(ops["meanImg"])


fig.add_axes([0.2, 0.65, 0.18, 0.18])
depth_selectivity.plot_example_fov(
    neurons_df=neurons_df_rf,
    ops=ops,
    stat=stat,
    ndepths=len(depths),
    col="rf_azi",
    cmap=cm.YlOrRd.reversed(),
    background_color=np.array([0, 0, 0]),
    fontsize_dict=fontsize_dict,
)

fig.add_axes([0.2, 0.5, 0.18, 0.18])
depth_selectivity.plot_example_fov(
    neurons_df=neurons_df_rf,
    ops=ops,
    stat=stat,
    ndepths=len(depths),
    col="rf_ele",
    cmap=cm.YlOrRd.reversed(),
    background_color=np.array([0, 0, 0]),
    fontsize_dict=fontsize_dict,
)

fig.add_axes([0.4, 0.5, 0.3, 0.3])
depth_selectivity.plot_example_fov(
    neurons_df=neurons_df_rf,
    ops=ops,
    stat=stat,
    ndepths=len(depths),
    col="preferred_depth_closedloop",
    cmap=cm.cool.reversed(),
    background_color=np.array([0, 0, 0]),
    fontsize_dict=fontsize_dict,
)

rf.plot_gradient_polar(
    session_df_all["azi_angle"], 
    nbins=36, 
    plot_x=0.75, 
    plot_y=0.7, 
    plot_width=0.1,
    plot_height=0.1,
    edgecolor='none',
    facecolor='royalblue',
    alpha=1,
)

rf.plot_gradient_polar(
    session_df_all["ele_angle"], 
    nbins=36, 
    plot_x=0.75, 
    plot_y=0.5, 
    plot_width=0.1,
    plot_height=0.1,
    edgecolor='none',
    facecolor='royalblue',
    alpha=1,
    fontsize_dict=fontsize_dict,
)
plt.savefig(SAVE_ROOT / "fig2b.pdf", dpi=300)
plt.savefig(SAVE_ROOT / "fig2b.png", dpi=300)

In [ ]:
# session_df_all = rf.calculate_rf_gradient_all_sessions(flexilims_session,
#                                        session_list=neurons_df_all.session.unique(),
#                                        neurons_df_all_aligned = neurons_df_all,
#                                        filename='rf_gradients.pkl',
#                                        conflicts='skip',
#                                        verbose=True,
#                                        )
# session_df_all.to_pickle(SAVE_ROOT / "rf_gradients_all_sessions.pkl")
session_df_all = pd.read_pickle(SAVE_ROOT / "rf_gradients_all_sessions.pkl")
session_df_all["azi_angle"] = np.degrees(np.arctan2(session_df_all["slope_y_azi_rf_sig"], -session_df_all["slope_x_azi_rf_sig"]))
session_df_all["ele_angle"] = np.degrees(np.arctan2(session_df_all["slope_y_ele_rf_sig"], -session_df_all["slope_x_ele_rf_sig"]))

## Preferred depth map


In [ ]:
from tqdm import tqdm

overview_img = roi_location.check_neurons_in_v1(neurons_df_all)
neurons_df_all["preferred_depth_corrected"] = neurons_df_all[
    "preferred_depth_closedloop"
] / np.sqrt(
    (np.sin(np.deg2rad(neurons_df_all["rf_azi"])) ** 2)
    * (np.cos(np.deg2rad(neurons_df_all["rf_ele"])) ** 2)
    + (np.sin(np.deg2rad(neurons_df_all["rf_ele"])) ** 2)
)

sig_neurons = (
    (neurons_df_all["iscell"] == True)
    & (neurons_df_all["depth_tuning_test_spearmanr_pval_closedloop"] < 0.05)
    & (neurons_df_all["depth_tuning_test_spearmanr_rval_closedloop"] > 0.1)
    & (neurons_df_all["v1"] == True)
    & (neurons_df_all["rf_sig"] == True)
)
neurons_df_all["log_preferred_depth"] = neurons_df_all[
    "preferred_depth_closedloop"
].apply(lambda x: np.log(x))
neurons_df_all["log_preferred_depth_corrected"] = neurons_df_all[
    "preferred_depth_corrected"
].apply(lambda x: np.log(x))
xrange = int(neurons_df_all["overview_x_aligned"].min()), int(
    neurons_df_all["overview_x_aligned"].max()
)
yrange = int(neurons_df_all["overview_y_aligned"].min()), int(
    neurons_df_all["overview_y_aligned"].max()
)
xx, yy = np.mgrid[xrange[0] : xrange[1] : 10, yrange[0] : yrange[1] : 10]
maps = {}
maps_alpha = {}
for icol, col in enumerate(
    ["rf_azi", "rf_ele", "log_preferred_depth", "log_preferred_depth_corrected"]
):
    # for every xx and yy, calculated the spatially weighted sum of the neurons as a function of distance
    zz = np.ravel(np.zeros(xx.shape))
    alpha = np.ravel(np.zeros(xx.shape))
    length_scale = 25
    for i, (x, y) in tqdm(enumerate(zip(np.ravel(xx), np.ravel(yy))), total=xx.size):
        weights = np.exp(
            -(
                (neurons_df_all[sig_neurons]["overview_x_aligned"] - x) ** 2
                + (neurons_df_all[sig_neurons]["overview_y_aligned"] - y) ** 2
            )
            / length_scale**2
        )
        alpha[i] = np.sum(weights)
        zz[i] = (
            np.dot(
                weights,
                neurons_df_all[sig_neurons][col].values,
            )
            / alpha[i]
        )
    maps[col] = np.reshape(zz, xx.shape).T
    maps_alpha[col] = np.reshape(alpha, xx.shape).T

In [ ]:
plt.figure(figsize=(18 / 2.54, 7 / 2.54))
plt.subplot(2, 4, 1)
plt.imshow(overview_img)
plt.xticks([])
plt.yticks([])
xlims = [50, overview_img.shape[1] - 50]
ylims = [50, overview_img.shape[0] - 50]
plt.xlim(xlims)
plt.ylim(ylims)
plt.gca().invert_yaxis()
for icol, col in enumerate(["rf_azi", "rf_ele", "log_preferred_depth_corrected"]):
    ax = plt.subplot(2, 4, 2 + icol)
    if col in ["log_preferred_depth_corrected", "log_preferred_depth"]:
        palette = cm.cool.reversed()
        vrange = [np.log(0.2), np.log(5)]
    else:
        palette = cm.YlOrRd.reversed()
        vrange = (
            neurons_df_all[sig_neurons][col].min(),
            neurons_df_all[sig_neurons][col].max(),
        )

    ax = sns.scatterplot(
        neurons_df_all[sig_neurons],
        x="overview_x_aligned",
        y="overview_y_aligned",
        hue=col,
        palette=palette,
        s=1,
        alpha=0.3,
        linewidth=0,
        legend=False,
        ax=ax,
        vmax=vrange[1],
        vmin=vrange[0],
    )
    norm = plt.Normalize(vmin=vrange[0], vmax=vrange[1])
    sm = plt.cm.ScalarMappable(cmap=palette, norm=norm)
    sm.set_array([])
    # Remove the legend and add a colorbar
    cbar = ax.figure.colorbar(sm, shrink=0.5, ticks=vrange)
    cbar.remove()
    ax.set_aspect("equal", adjustable="box")
    plt.xlabel("")
    plt.ylabel("")
    plt.xlim(xlims)
    plt.ylim(ylims)
    ax.invert_yaxis()
    ax.invert_xaxis()

    plt.xticks([])
    plt.yticks([])

    alpha = maps_alpha[col]
    max_alpha = alpha.max()
    alpha = alpha / max_alpha * 5
    alpha = np.clip(alpha, 0, 1)
    plt.subplot(2, 4, 6 + icol)
    plt.imshow(
        maps[col],
        extent=(xrange[0], xrange[1], yrange[0], yrange[1]),
        origin="lower",
        alpha=alpha,
        cmap=palette,
        vmin=vrange[0],
        vmax=vrange[1],
    )
    cbar = plt.colorbar(shrink=0.5, ticks=vrange)
    if col in ["log_preferred_depth_corrected", "log_preferred_depth"]:
        cbar.ax.set_yticklabels(["<0.2", ">5"])
    cbar.ax.tick_params(labelsize=fontsize_dict["tick"])
    plt.xlim(xlims)
    plt.ylim(ylims)
    plt.gca().invert_yaxis()
    plt.gca().invert_xaxis()
    plt.xticks([])
    plt.yticks([])
    plt.axis("off")

plt.savefig(SAVE_ROOT / "v1_map.pdf", dpi=300)

In [ ]:
fig = plt.figure(figsize=(18 / 2.54, 10 / 2.54))
fontsize_dict = {"title": 7, "label": 7, "tick": 6, "legend": 5}

neurons_df_all["rf_azi_jittered"] = neurons_df_all["rf_azi"] + np.random.uniform(
    -2, 2, neurons_df_all.shape[0]
)
neurons_df_all["rf_ele_jittered"] = neurons_df_all["rf_ele"] + np.random.uniform(
    -2, 2, neurons_df_all.shape[0]
)

sig_neurons = (
    (neurons_df_all["rf_sig"] == True)
    & (neurons_df_all["iscell"] == True)
    & (neurons_df_all["depth_tuning_test_spearmanr_pval_closedloop"] < 0.05)
    & (neurons_df_all["depth_tuning_test_spearmanr_rval_closedloop"] > 0.1)
)
neurons_df_sig = neurons_df_all[sig_neurons]
xrange = 17.5, 117.5
yrange = -37.5, 37.5
xx, yy = np.mgrid[xrange[0] : xrange[1] + 5 : 5, yrange[0] : yrange[1] + 5 : 5]

# for every xx and yy, calculated the spatially weighted sum of the neurons as a function of distance
ranges = [[-np.Inf, 0.2], [0.2, 1], [1, np.Inf]]

zz_near = np.ravel(np.zeros(xx.shape))
zz_mid = np.ravel(np.zeros(xx.shape))
zz_far = np.ravel(np.zeros(xx.shape))
length_scale = 5
col = "preferred_depth_corrected"


def format_plot(ax):
    sns.despine(offset=2, ax=ax)
    ax.set_xticks([30, 60, 90, 120])
    ax.set_yticks([-40, 0, 40])
    ax.tick_params(labelsize=fontsize_dict["tick"], length=2, direction="out")


def calculate_zz(condition):
    zz = np.ravel(np.zeros(xx.shape))
    for i, (x, y) in tqdm(enumerate(zip(np.ravel(xx), np.ravel(yy))), total=xx.size):
        idx = (neurons_df_sig["rf_azi"] == x) & (neurons_df_sig["rf_ele"] == y)
        if np.sum(idx) < 20:
            zz[i] = np.nan
        else:
            zz[i] = np.mean(condition[idx])
    return np.reshape(zz, xx.shape).T


im_extent = (xrange[0] - 2.5, xrange[1] + 2.5, yrange[0] - 2.5, yrange[1] + 2.5)

median_depth = np.ravel(np.zeros(xx.shape))
for i, (x, y) in tqdm(enumerate(zip(np.ravel(xx), np.ravel(yy))), total=xx.size):
    idx = (neurons_df_sig["rf_azi"] == x) & (neurons_df_sig["rf_ele"] == y)
    if np.sum(idx) < 20:
        median_depth[i] = np.nan
    else:
        median_depth[i] = np.median(neurons_df_sig[idx][col])
median_depth = np.reshape(median_depth, xx.shape).T

vrange = [np.log(0.05), np.log(6.4)]

# ax = plt.subplot2grid((6, 4), (0, 0), rowspan=3, colspan=2, aspect="equal")
# norm = plt.Normalize(vmin=vrange[0], vmax=vrange[1])
# ax = sns.scatterplot(
#     neurons_df_sig,
#     x="rf_azi_jittered",
#     y="rf_ele_jittered",
#     hue=np.log(neurons_df_sig[col]),
#     palette="cool_r",
#     s=1,
#     alpha=0.3,
#     linewidth=0,
#     legend=False,
#     ax=ax,
#     hue_norm=norm,
# )

# sm = plt.cm.ScalarMappable(cmap="cool_r", norm=norm)
# sm.set_array([])
# # Remove the legend and add a colorbar
# cbar = ax.figure.colorbar(sm, shrink=0.5)
# cbar.ax.set_yticks(np.log([0.2, 1, 5]))
# cbar.ax.set_yticklabels([0.2, 1, 5])
# cbar.ax.tick_params(labelsize=fontsize_dict["tick"])
# plt.ylabel("")
# plt.xlabel("")
# ax.set_xlim([15, 120])
# ax.set_ylim([-40, 40])
# format_plot(ax)


ax = fig.add_axes([0.05, 0.1, 0.4, 0.4])
im=ax.imshow(
    np.log(median_depth),
    extent=im_extent,
    origin="lower",
    cmap="cool_r",
    vmin=vrange[0],
    vmax=vrange[1],
)
plt.xlabel("RF azimuth (degrees)", fontsize=fontsize_dict["label"])
plt.ylabel("RF elevation (degrees)", fontsize=fontsize_dict["label"])
format_plot(ax)
sns.despine()

ax_pos = ax.get_position()
cbar_ax = plt.gcf().add_axes(
    [
        ax_pos.x0 + ax_pos.width*1.05,
        ax_pos.y0,
        0.01,
        ax_pos.height/2,
    ]
)
cbar = plt.colorbar(im, cax=cbar_ax)
cbar.ax.set_yticks(np.log([0.2, 1, 5]))
cbar.ax.set_yticklabels([0.2, 1, 5])
cbar.ax.tick_params(labelsize=fontsize_dict["tick"], pad=2)

# Plot scatter map for each depth range
for i, (r, text) in enumerate(zip(ranges, ["<20 cm", "20-100 cm", ">100 cm"])):
    ax = fig.add_axes([0.5 + 0.15*i, 0.35, 0.1, 0.15])
    idx = (neurons_df_sig[col].values > r[0]) & (neurons_df_sig[col].values <= r[1])
    ax = sns.scatterplot(
        neurons_df_sig[idx],
        x="rf_azi_jittered",
        y="rf_ele_jittered",
        hue=np.log(neurons_df_sig[idx][col]),
        palette="cool_r",
        s=1,
        alpha=0.2,
        linewidth=0,
        legend=False,
        ax=ax,
        hue_norm=norm,
    )
    if i == 0:
        plt.ylabel("RF elevation (degrees)", fontsize=fontsize_dict["label"])
    else:
        plt.ylabel("")
    plt.xlabel("")
    ax.set_xlim([15, 120])
    ax.set_ylim([-40, 40])
    format_plot(ax)

    ax.text(120, 45, text, ha="right", fontsize=fontsize_dict["label"])
    # Plot binned map for each depth range
    zz = calculate_zz(idx)
    ax = fig.add_axes([0.5 + 0.15*i, 0.1, 0.1, 0.15])
    im = plt.imshow(
        zz,
        extent=im_extent,
        origin="lower",
        cmap="Reds",
        vmin=0,
        vmax=0.6,
    )
    format_plot(ax)
    if i == 1:
        plt.xlabel("RF azimuth (degrees)", fontsize=fontsize_dict["label"])
    if i == 2:
        ax_pos = ax.get_position()
        cbar_ax = plt.gcf().add_axes(
            [
                ax_pos.x0 + ax_pos.width*1.1,
                ax_pos.y0,
                0.005,
                ax_pos.height/2,
            ]
        )
        cbar = plt.colorbar(im, cax=cbar_ax, ticks=[0, 0.6])
        cbar.ax.tick_params(labelsize=fontsize_dict["tick"], pad=2)

plt.tight_layout()
plt.savefig(SAVE_ROOT / "rf_position_corrected.pdf", dpi=300)

In [ ]:
xrange = 27.5, 117.5
yrange = -37.5, 37.5
xx, yy = np.mgrid[xrange[0] : xrange[1] + 5 : 5, yrange[0] : yrange[1] + 5 : 5]
im_extent = (xrange[0] - 2.5, xrange[1] + 2.5, yrange[0] - 2.5, yrange[1] + 2.5)

correction = np.log2(
    1
    / np.sqrt(
        (np.sin(np.deg2rad(xx)) ** 2) * (np.cos(np.deg2rad(yy)) ** 2)
        + (np.sin(np.deg2rad(yy)) ** 2)
    )
)

plt.imshow(correction.T, extent=im_extent, origin="lower")
plt.colorbar()

In [ ]:
from scipy.stats import spearmanr
from sklearn.linear_model import LinearRegression

sig_neurons = (
    (neurons_df_all["iscell"] == True)
    & (neurons_df_all["depth_tuning_test_spearmanr_pval_closedloop"] < 0.05)
    & (neurons_df_all["overview_x_aligned"].isna() == False)
    & (neurons_df_all["v1_mask"] == True)
    & (neurons_df_all["rf_sig"] == True)
)

neurons_df_all["log_preferred_depth_corrected"] = neurons_df_all[
    "preferred_depth_corrected"
].apply(lambda x: np.log(x))

n_boot = 1000
corrs = {
    "rf_azi": [],
    "rf_ele": [],
    "z_position": [],
    "overview_x_aligned": [],
    "overview_y_aligned": [],
}
np.random.seed(0)
coefs = []
coefs_corrected = []
for i in tqdm(range(n_boot)):
    sample = common_utils.bootstrap_sample(
        neurons_df_all[sig_neurons], ["mouse", "session"]
    )
    for col in corrs:
        corrs[col].append(
            spearmanr(
                neurons_df_all.loc[sample, col],
                neurons_df_all.loc[sample, "preferred_depth_closedloop"],
            ).statistic
        )
    # determine how col depends on rf_azi and rf_ele
    X = neurons_df_all.loc[sample, ["rf_azi", "rf_ele"]].values
    y = neurons_df_all.loc[sample, "log_preferred_depth"].values
    reg = LinearRegression().fit(X, y)
    coefs.append(reg.coef_)

    y = neurons_df_all.loc[sample, "log_preferred_depth_corrected"].values
    reg = LinearRegression().fit(X, y)
    coefs_corrected.append(reg.coef_)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import circmean

coefs = np.array(coefs)
coefs_corrected = np.array(coefs_corrected)
plt.figure(figsize=(10, 8))
ax = plt.subplot(121, polar=True)
gradient = np.arctan2(coefs[:, 1], coefs[:, 0])
ax.hist(
    gradient,
    bins=50,
    color="black",
    edgecolor="black",
)
plt.title("Gradient of RF position vs preferred depth")
ax.set_xticks(np.linspace(0, 2 * np.pi, 4, endpoint=False))
ax.set_xticklabels(["Temporal", "Up", "Nasal", "Down"])

mean_angle = circmean(gradient[: len(gradient) // 2])

# Calculate the circular distance from the mean for each angle
distances = np.sort(
    (gradient[len(gradient) // 2 :] - mean_angle + np.pi) % (2 * np.pi) - np.pi
)

# Calculate the 25th and 75th percentiles of these distances
print(np.quantile(distances, [0.025, 0.975]))
print(np.mean(np.abs(distances) > np.pi / 2))
ax = plt.subplot(122, polar=True)
ax.hist(
    np.arctan2(coefs_corrected[:, 1], coefs_corrected[:, 0]),
    bins=50,
    color="black",
    edgecolor="black",
)
plt.title("Gradient of RF position vs corrected preferred depth")
ax.set_xticks(np.linspace(0, 2 * np.pi, 4, endpoint=False))
ax.set_xticklabels(["Temporal", "Up", "Nasal", "Down"])
plt.tight_layout()

In [ ]:
col = "preferred_depth_corrected"
plt.figure(figsize=(15, 4))
for i, col in enumerate(corrs):
    plt.subplot(1, 5, i + 1)
    plt.hist(corrs[col], bins=20, color="k")
    # plot a line at 0.025 and 0.975 quantiles
    plt.axvline(np.quantile(corrs[col], 0.025), c="r")
    plt.axvline(np.quantile(corrs[col], 0.975), c="r")
    # find proportion of values on the other side of 0 from the median
    p = np.mean(np.array(corrs[col]) > 0)
    if p > 0.5:
        p = 1 - p
    p *= 2
    plt.title(f"{col} p = {p:.3f}")

In [ ]:
# MINOR:
# - add location on overview & scalebars (can highlight the neurons above)
# - DONE: change the near cell example to 591
# - DONE: change colormap for azimuth/elevation to plasma, viridis etc summer autumn, BuYl

# CHECK:
# - DONE: double check if significant RF cells include optic flow near cells

# MAJOR:
# - summary of all sessions: compute the distance between n nearest neighbors vs n random neighbors. sklearn.neighbors.KDTree

# Other:
# - R2 distribution, add non-linear fit (no need to do it yet);

## SUPP


In [ ]:
# Fig.2E Distribution of azimuth/elevation of depth-selective neurons in 2 example imaging sessions
# Load data for plotting center of RFs
sessions_dist_exp = ["PZAH6.4b_S20220419", "PZAH6.4b_S20220426"]
results_dist_exp = rf.get_rf_results(project, sessions_dist_exp)

fig = plt.figure()
rf.plot_rf_centers(
    fig=fig,
    results=results_dist_exp,
    colors=["r", "b"],
    ndepths=5,
    frame_shape=(16, 24),
    n_stds=5,
    plot_x=0.3,
    plot_y=0,
    plot_width=0.3,
    plot_height=0.3,
    fontsize_dict={"title": 15, "label": 10, "tick": 5},
)